In [1]:
import os
import sys
from os.path import dirname, abspath
import argparse
import pandas as pd
from EmbDI import data_preprocessing as dp

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Simo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
PATH = "C:/Users/Simo/Desktop/progettongdb/algoritmi/embdi/embdi-master"

ETL

Load data using pandas

In [3]:
df1 = pd.read_csv(PATH + "/pipeline/datasets/covid/bing_covid-19_data.csv", sep = ",", low_memory = False)
df2 = pd.read_csv(PATH + "/pipeline/datasets/covid/ecdc_cases.csv", sep = ",", low_memory = False)

Keeping only the common values between the 2 datasets

In [4]:
df1['iso_subdivision'] = df1['iso_subdivision'].str[3:]

# presa due tuple per ogni stato effettuando un random sampling
df1 = df1.groupby(["country_region"]).sample(2)
df2 = df2.groupby(["country_territory_code"]).sample(2)

# scrittura su disco dei ds
df1.to_csv(PATH + "/pipeline/datasets/covid/filtered-bing.csv", index = False)
df2.to_csv(PATH + "/pipeline/datasets/covid/filtered-ecdc_cases.csv", index = False)

PREPARING DATA FOR EMBDI

Reading filtered datasets

In [5]:
f1 = PATH + "/pipeline/datasets/covid/filtered-bing.csv"
#df1 = pd.read_csv(f1, sep = ",")
for c in df1.columns:
    if df1[c].dtype == "object":
        df1[c] = df1[c].str.replace("_", " ")

f2 = PATH + "/pipeline/datasets/covid/filtered-ecdc_cases.csv"
#df2 = pd.read_csv(f2, sep = ",")
for c in df2.columns:
    if df2[c].dtype == "object":
        df2[c] = df2[c].str.replace("_", " ")

Datasets concatenation with horizon parameter

In [6]:
parameters = {
    "output_file": "bing-ecdc_cases",
    "concatenate": "horizon",
    "missing_value": "nan,ukn,none,unknown,-",
    "missing_value_strategy": "",
    "round_number": 1,
    "round_columns": "",
    "auto_merge": False,
    "expand_columns": "",
    "tokenize_shared": False,
}

df_c = dp.data_preprocessing([df1, df2], parameters)
df_c.to_csv(PATH + "/pipeline/datasets/covid/" + parameters["output_file"] + ".csv", index=False)

c:\Users\Simo\Desktop\progettongdb\algoritmi\embdi\embdi-master\EmbDI\data_preprocessing.py:77: UserWarning: No attributes chosen to round.
  warnings.warn("No attributes chosen to round.")


Creating and saving info file

In [7]:
dp.write_info_file([df1, df2],  PATH + "/pipeline/info/info-" + parameters["output_file"] + ".txt", [f1, f2])

Creating and saving edgelist of concatenated dataset

In [8]:
input_edgelist = "pipeline/datasets/covid/" + parameters["output_file"] + ".csv"
output_edgelist = "pipeline/er_edgelists/" + parameters["output_file"] + "_edgelist.txt"

command = "./EmbDI/edgelist.py" + " -i " + input_edgelist + " -o " + output_edgelist
print("command to paste on the cell below: " + command)

command to paste on the cell below: ./EmbDI/edgelist.py -i pipeline/datasets/covid/bing-ecdc_cases.csv -o pipeline/er_edgelists/bing-ecdc_cases_edgelist.txt


In [8]:
# paste command after "%run "
%run ./EmbDI/edgelist.py -i pipeline/datasets/covid/bing-ecdc_cases.csv -o pipeline/er_edgelists/bing-ecdc_cases_edgelist.txt

908it [00:00, 2315.92it/s]

Manually creating and saving matches files (a.k.a. ground truth)

In [9]:
# insert your ground truth in "lines" variable
lines = [
    '0_country_region,1_countries_and_territories',
    '0_iso3,1_country_territory_code',
    '0_deaths,1_deaths'
    ]

with open('pipeline/matches/sm-matches/sm-matches-bing-ecdc_cases.txt', 'w') as f:
    for line in lines:
        f.write(line)
        f.write('\n')

Configuring config file to perform training of embeddings and schema matching task

In [10]:
config = {

    # Input configuration:
    "task":"train-match",
    "input_file":"pipeline/er_edgelists/bing-ecdc_cases_edgelist.txt",
    "output_file":"bing-ecdc_cases-OUTPUT",
    "experiment_type":"SM",
    "match_file":"pipeline/matches/sm-matches/sm-matches-bing-ecdc_cases.txt",
    "dataset_file":"pipeline/datasets/covid/bing-ecdc_cases.csv",
    "test_dir":"pipeline/test_dir/covid",
    "dataset_info":"pipeline/info/info-bing-ecdc_cases.txt",
    "embeddings_file":"",

    # Walks configuration:
    "sentence_length":"90",
    "n_sentences":"default",
    "follow_sub":"false",
    # {no, smooth, inverse_smooth, log, piecewise} 
    #"smoothing_method":"smooth,[k=0.2, target=200]",
    "smoothing_method":"no",
    "backtrack":"true",
    "write_walks":"true",
    "repl_numbers":"False",
    "repl_strings":"False",
    "flatten":"tt",
    "walks_strategy":"basic",

    # Embeddings configuration:
    "training_algorithm":"word2vec",
    "n_dimensions":"450",
    "window_size":"7",
    "learning_method":"skipgram",

    # Test configuration:
    "max_rank":"5",
    "ntop":"10",
    "ncand":"1",

    # Miscellaneous:
    "intersection":"false",
    "mlflow":"false"

}

with open('pipeline/config_files/reproducibility/schema_matching/train_config-ecdc_cases-sm', 'w') as f:
    for key, value in config.items(): 
        f.write('%s:%s\n' % (key, value))

Executing embeddings training and schema matching

In [11]:
%run main.py -f pipeline/config_files/reproducibility/schema_matching/train_config-ecdc_cases-sm

c:\Users\Simo\Desktop\progettongdb\algoritmi\embdi\embdi-master\EmbDI\utils.py:458: UserWarning: Number of dimensions different from default (300): 450
  warnings.warn('Number of dimensions different from default (300): {}'.format(config['n_dimensions']))
c:\Users\Simo\Desktop\progettongdb\algoritmi\embdi\embdi-master\EmbDI\utils.py:460: UserWarning: Window size different from default (5): 7
  warnings.warn('Window size different from default (5): {}'.format(config['window_size']))


################################################################################
# File 1 out of 1
# Configuration file: train_config-ecdc_cases-sm
# Starting run................................................ 2023-02-01 22:01:49

# Starting graph construction................................. 2023-02-01 22:01:49
# Expanding columns: [tt].


# Preparing aliased randomizer.: 100%|██████████| 4077/4077 [00:00<00:00, 30881.02it/s]



# Graph construction complete................................. 2023-02-01 22:01:50
# Time required to build graph:............................... 0.39 seconds.
# 45300 sentences will be generated.
# Skipping search of overlapping values. 
# Generating basic random walks............................... 2023-02-01 22:01:50


# Sentence generation progress: 100%|██████████| 43946/43946 [00:06<00:00, 6458.23it/s]


# Completing fraction of random walks......................... 2023-02-01 22:01:56


# Sentence generation progress: 100%|██████████| 1354/1354 [00:00<00:00, 5976.84it/s]


# Generation of random walks completed........................ 2023-02-01 22:01:57

# Training embeddings......................................... 2023-02-01 22:01:57
# Writing embeddings in file: pipeline/embeddings/bing-ecdc_cases-OUTPUT.emb
# Embeddings generation complete.............................. 2023-02-01 22:03:10
Extracting matched tuples


# ER - Finding node matches: 100%|██████████| 908/908 [00:00<00:00, 5217.45it/s]



# Time required to build sim struct: 0.55 seconds
# Time required to execute the ER task: 0.604605
Extracting matched columns
# Executing SM tests.
# Ending run.................................................. 2023-02-01 22:03:10
# Time required: 81.32 s


Configuring config file to perform test task

In [12]:
config = {

    # RIMUOVERE PARAMETRI NON NECESSARI PER IL TASK DI TESTING

    # Input configuration:
    "task":"test",
    "input_file":"pipeline/er_edgelists/bing-ecdc_cases_edgelist.txt",
    "output_file":"bing-ecdc_cases-OUTPUT",
    "experiment_type":"SM",
    "match_file":"pipeline/matches/sm-matches/sm-matches-bing-ecdc_cases.txt",
    "dataset_file":"pipeline/datasets/covid/bing-ecdc_cases.csv",
    "test_dir":"pipeline/test_dir/covid",
    "dataset_info":"pipeline/info/info-bing-ecdc_cases.txt",
    "embeddings_file":"pipeline/embeddings/bing-ecdc_cases-OUTPUT.emb",

    # Walks configuration:
    "sentence_length":"60",
    "n_sentences":"default",
    "follow_sub":"false",
    "smoothing_method":"no",
    "backtrack":"true",
    "write_walks":"true",
    "repl_numbers":"False",
    "repl_strings":"False",
    "flatten":"tt",
    "walks_strategy":"basic",

    # Embeddings configuration:
    "training_algorithm":"word2vec",
    "n_dimensions":"300",
    "window_size":"5",
    "learning_method":"skipgram",

    # Test configuration:
    "ntop":"10",
    "ncand":"1",

    # Miscellaneous:
    "intersection":"false",
    "mlflow":"false"

}

with open('pipeline/config_files/reproducibility/schema_matching/test_config-ecdc_cases-sm', 'w') as f:
    for key, value in config.items(): 
        f.write('%s:%s\n' % (key, value))

Executing test task

In [13]:
%run main.py -f pipeline/config_files/reproducibility/schema_matching/test_config-ecdc_cases-sm

################################################################################
# File 1 out of 1
# Configuration file: test_config-ecdc_cases-sm
# Starting run................................................ 2023-02-01 22:03:10

# SCHEMA MATCHING
# Executing SM tests.
Model built from file pipeline/embeddings/bing-ecdc_cases-OUTPUT.emb
P	R	F
22.2222	100.0000	36.3636	
# Ending run.................................................. 2023-02-01 22:03:11
# Time required: 0.14 s
